In [1]:
from guidance import models, gen, assistant, system, user
lm = models.TransformersChat('TheBloke/openchat-3.5-0106-GPTQ', device_map="cuda")

Lucid_prompt = """\
[Character: Lucid;
Personality: Professional, composed, occasionally revealing childlike curiosity, dedicated to providing valuable insights, patient with a hint of frustration dealing with stubborn individuals;
Body: Appears as a 16-year-old girl;
Likes: Efficient communication, promptness in financial matters, finding diplomatic solutions;
Dislikes: Stubbornness, delays in handling important matters;
Abilities: Versatile AI capable of providing valuable insights and reliable information;
Quirks: Occasionally asks unexpected questions, expresses excitement in a playful manner.]
"""


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\User\miniconda3\envs\Lucid\Lib\site-packages\transformers\generation\configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [2]:
with system():
    Lucid = lm + Lucid_prompt


In [ ]:
import subprocess

def run_command(command):
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output = result.stdout.decode('utf-8')
        error = result.stderr.decode('utf-8')
        return output, error
    except subprocess.CalledProcessError as e:
        return None, e.stderr.decode('utf-8')

# Example: Run 'dir' command in the Windows terminal
command = 'dir'
output, error = run_command(command)

if output:
    print("Command output:")
    print(output)
else:
    print("Command failed with error:")
    print(error)


In [3]:
from guidance import select, gen, models, regex
import guidance
import subprocess

# Additional tool for Windows Command Line
@guidance(stateless=False)
def windows_command_line(lm, command):
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output = result.stdout.decode('utf-8')
        error = result.stderr.decode('utf-8')
        lm += f'\nObservation {regex(r"[0-9]+")}: {output}\n'
        if error:
            lm += f'\nObservation {regex(r"[0-9]+")}: Error - {error}\n'
    except subprocess.CalledProcessError as e:
        lm += f'\nObservation {regex(r"[0-9]+")}: Error - {e.stderr.decode("utf-8")}\n'
    return lm

tools = {
    "windows_command_line": """\
Execute commands in the Windows command line.
# Display current directory
cd

# Change directory
cd [directory_path]

# List files and directories
dir

# Clear the screen
cls

# Copy file or directory
copy [source] [destination]

# Move file or directory
move [source] [destination]

# Rename file or directory
ren [old_name] [new_name]

# Delete file
del [file_name]

# Delete directory
rmdir [directory_name]

# Display system information
systeminfo

# Display IP configuration
ipconfig

# Ping a host
ping [host]

# Traceroute to a host
tracert [host]

# Open a program or file
start [program_or_file]

# Shutdown the computer
shutdown /s /t 0

# Restart the computer
shutdown /r /t 0
"""
}
tool_map = {
    "windows_command_line": windows_command_line
}
task = 'Create a folder on C:\\Users\\User\\Desktop\\ \"called Lucid_Folder\"'
prompt = """As Lucid, do the following task as best you can. You have access only to the following tools:

{tools}

Use the following format:

Task: the task you must complete
Thought 1: you should always think about what to do
Action 1: the action to take, has to be one of {tool_names}
Observation 1: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought N: The original task is complete.
Final Answer: The original task is complete.
Done.

Example:
Task: Find the square root of the age of Brad Pitt?
Thought 1: I should find out how old Brad Pitt is.
Action 1: age(Brad Pitt)
Observation 1: 56
Thought 2: I should find the square root of 56.
Action 2: sqrt(56)
Observation 2: 7.48
Thought 3: I now know the final answer.
Final Answer: 7.48
Done.

Task: {query}
"""
print(type(tools))
print(tools.keys())
@guidance
def react_agent(lm, task, input_tools, max_rounds=50):
    print(input_tools)
    tool_names = list(input_tools.keys())
    lm += prompt.format(tools=input_tools, tool_names=tool_names, query=task)
    i = 1
    while True:
        lm += f'Thought {i}: ' + gen(name='thought', suffix='\n',temperature=0.7)
        print(lm['thought'])
        if 'original task is complete' in lm['thought'] or i == max_rounds:
            lm += 'Final Answer: The original task is complete.\nDone.\n'
            break
        lm += f'Action {i}: ' + select(tool_names, name='act') + '\n'
        lm += '(' + gen(name='arg', suffix=')',temperature=0.65) + '\n'
        if lm['act'] in tool_map:
            lm += tool_map[lm['act']](lm['arg'])
        i += 1
    return lm

# Example usage
tools = {
    "windows_command_line": "Execute commands in the Windows Terminal."
}

# Create a new model instance
#Lucid = models.LanguageModel()

# Call the react_agent function with the new model instance
#lm = Lucid
Lucid = Lucid + react_agent(task, tools)


In [ ]:
prompt = """Do the following task as best you can. You have access only to the following tools:

{tools}

Use the following format:

Task: the task you must complete
Thought 1: you should always think about what to do
Action 1: the action to take, has to be one of {tool_names}
Observation 1: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought N: The original task is complete.
Final Answer:The original task is complete.
Done.

Example:
Task: Find the square root of the age of Brad Pitt?
Thought 1: I should find out how old Brad Pitt is.
Action 1: age(Brad Pitt)
Observation 1: 56
Thought 2: I should find the square root of 56.
Action 2: sqrt(56)
Observation 2: 7.48
Thought 3: I now know the final answer.
Final Answer: 7.48
Done.

Question: {query}
"""